In [1]:
!pip install transformers


In [2]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import pipeline
import torch
import numpy as np

/home/marcelino.maita/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-10-24 17:38:55.728447: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-24 17:38:55.801290: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2025-10-24 17:39:00.892038: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [3]:
RANDOM_STATE = 42
path_DATA = "../../../data"
csv_path = f"{path_DATA}/spotify_dataset_sin_duplicados_4.csv"

TESTING = False

if TESTING:
    NROWS = 50
else:
    NROWS = None
    


In [4]:
def get_array(path):
    with open(path, "r") as f:
        array_ = json.load(f)  
    return array_

def get_song_and_target(csv_path, faltantes_path = "faltantes_according_token.json", sample_size=None):
    df = pd.read_csv(csv_path, nrows=sample_size)
    indices_to_remove = get_array(faltantes_path)
    if indices_to_remove[-1]>df.shape[0]:
        print("You are executing in testing way")
    else:
        df = df.drop(indices_to_remove).reset_index(drop=True)
    X = df['text']
    df['Explicit_binary'] = (df['Explicit'].str.lower() == 'yes').astype(int)
    y = df['Explicit_binary']
    return X, y

In [5]:
X,y = get_song_and_target(csv_path,"faltantes_according_token.json", sample_size=NROWS)

print(X.shape)
print(y.shape)

(108125,)
(108125,)


MoritzLaurer/deberta-v3-large-zeroshot-v2.0  
https://huggingface.co/MoritzLaurer/deberta-v3-large-zeroshot-v2.0

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=RANDOM_STATE
)

print("X_train.shape:", X_train.shape)
print("y_test.shape:", y_train.shape)


hypothesis_template = "This lyric contains {} content"
classes_verbalized = ["explicit", "not explicit"]

device = 0 if torch.cuda.is_available() else -1

zeroshot_classifier = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0",
    device=device
)

train_scores = []
for lyric in X_train:
    output = zeroshot_classifier(
        lyric,
        classes_verbalized,
        hypothesis_template=hypothesis_template,
        multi_label=False
    )
    score_explicit = output['scores'][output['labels'].index('explicit')]
    # print(output['labels'])
    # print(output['scores'])
    # print(score_explicit)
    train_scores.append(score_explicit)
    

train_scores = np.array(train_scores)

X_train.shape: (86500,)
y_test.shape: (86500,)


Device set to use cuda:0


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [7]:
thresholds = np.arange(0.1, 0.9, 0.05)
best_threshold = 0.5
best_f1 = 0

for t in thresholds:
    preds_binary = (train_scores >= t).astype(int)
    f1 = f1_score(y_train, preds_binary)
    # print(f"Threshold {t:.2f} -> F1: {f1:.4f}")
    if f1 > best_f1:
        best_f1 = f1
        best_threshold = t

print("Mejor threshold:", best_threshold, "con F1:", best_f1)

Mejor threshold: 0.8500000000000002 con F1: 0.6483344790950566


In [8]:
test_scores = []
for lyric in X_test:
    output = zeroshot_classifier(
        lyric,
        classes_verbalized,
        hypothesis_template=hypothesis_template,
        multi_label=False
    )
    score_explicit = output['scores'][output['labels'].index('explicit')]
    test_scores.append(score_explicit)

test_scores = np.array(test_scores)
test_preds = (test_scores >= best_threshold).astype(int)

# Métricas
f1_test = f1_score(y_test, test_preds)
accuracy_test = accuracy_score(y_test, test_preds)
precision_test = precision_score(y_test, test_preds)
recall_test = recall_score(y_test, test_preds)

print(f"Accuracy en test: {accuracy_test:.8f}")
print(f"Precision en test: {precision_test:.8f}")
print(f"Recall en test: {recall_test:.8f}")
print(f"F1 en test: {f1_test:.8f}")

Accuracy en test: 0.80097110
Precision en test: 0.56172483
Recall en test: 0.75484496
F1 en test: 0.64412105


## Prueba con treshold 0.5

In [9]:
best_threshold = 0.5
print("Probando con: ",best_threshold )
test_scores = []
for lyric in X_test:
    output = zeroshot_classifier(
        lyric,
        classes_verbalized,
        hypothesis_template=hypothesis_template,
        multi_label=False
    )
    score_explicit = output['scores'][output['labels'].index('explicit')]
    test_scores.append(score_explicit)

test_scores = np.array(test_scores)
test_preds = (test_scores >= best_threshold).astype(int)

# Métricas
f1_test = f1_score(y_test, test_preds)
accuracy_test = accuracy_score(y_test, test_preds)
precision_test = precision_score(y_test, test_preds)
recall_test = recall_score(y_test, test_preds)

print(f"Accuracy en test: {accuracy_test:.8f}")
print(f"Precision en test: {precision_test:.8f}")
print(f"Recall en test: {recall_test:.8f}")
print(f"F1 en test: {f1_test:.8f}")

Probando con:  0.5


Accuracy en test: 0.77553757
Precision en test: 0.51920181
Recall en test: 0.80174419
F1 en test: 0.63025594
